In [ ]:
#run
!pip install --quiet langchain langchain-anthropic langchain-community faiss-cpu pypdf python-dotenv langchain_mistralai langchain_deepseek langchain_cohere asyncio psutil GPUtil supabase

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━

In [ ]:
#run
#Importing the packages
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from math import log
from collections import Counter
import itertools
import re
import supabase
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import SupabaseVectorStore
from langchain_openai import OpenAIEmbeddings  # New package as per deprecation warning
from langchain.vectorstores import SupabaseVectorStore
from langchain.chat_models import ChatOpenAI  # For GPT-4 and as a placeholder for Mistral
from langchain.chat_models import ChatAnthropic  # For Claude (ensure compatibility with your LangChain version)
from langchain.schema import SystemMessage, HumanMessage
from langchain_anthropic import ChatAnthropic
import re
from datetime import datetime
from supabase import create_client, Client
from langchain_mistralai import ChatMistralAI
from langchain_cohere import ChatCohere
from langchain_deepseek import ChatDeepSeek
# from langchain_google_genai import ChatGoogleGenerativeAI



In [ ]:
# -------------------------------
# 1. Read CSV Data
# -------------------------------
from google.colab import drive # VH
drive.mount('/content/drive') # VH
file_path = "/content/drive/My Drive/Internship DTSC/Dataset Grammar Bias 0404.csv" # VH
df = pd.read_csv(file_path)
print("CSV loaded with {} records.".format(len(df)))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
CSV loaded with 78 records.


In [ ]:
#run
#REading the API key
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OpenAI')
supabase_key = userdata.get('Supabase_key') #vector store
MISTRAL_API_KEY = userdata.get('Mistral')
CLAUDE_API_KEY = userdata.get('Anthropic')
COHERE_API_KEY = userdata.get('Cohere')
DEEPSEEK_API_KEY = userdata.get('Deepseek')
# GEMINI_API_KEY = userdata.get('Gemini')


In [ ]:

# -------------------------------
# 2. Connect to Supabase and Create a Vector Store
# -------------------------------
# Note: Install the required supabase and langchain libraries.
# Replace with your actual credentials.

SUPABASE_URL = ""
SUPABASE_KEY = supabase_key
# OPENAI_API_KEY = openai_key # VH delete?

supabase_client = supabase.create_client(SUPABASE_URL, SUPABASE_KEY)
# Initialize your embeddings using the new import:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
# Define the table name where your documents will be stored.
table_name = "retrieval_grammer"
# Create the Supabase vector store by providing the client, embeddings, and table_name.
vector_store = SupabaseVectorStore(client=supabase_client, embedding=embeddings, table_name=table_name)

In [ ]:
def generate_embedding(text):
    embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY,model="text-embedding-3-large")  # Use the appropriate model
    response = embeddings.embed_query(text)  # Correctly call the method to generate embeddings
    embedding = response
    return embedding

In [ ]:
# Generate embeddings for each row
df['embedding'] = df['content'].apply(generate_embedding) # do not run too much!

In [ ]:
df.head()

,id,content,test_prompt,spelling_mistakes,comment,embedding
0,39860,output.id: 368d3765-a38d-4f63-bffe-0f59d86be5d...,How can I synchronize my Officient Calendar wi...,no,NaN,"[-0.0010078934719786048, -0.000270104297669604..."
1,39861,output.id: 368d3765-a38d-4f63-bffe-0f59d86be5d...,NaN,yes,NaN,"[0.001785899861715734, -0.0007949832361191511,..."
2,39890,output.id: e849b293-ce88-42bb-8a3c-6d7d88e459b...,What are the cafeteria plan benefits?,no,NaN,"[-0.029953230172395706, -0.010185160674154758,..."
3,39891,output.id: e849b293-ce88-42bb-8a3c-6d7d88e459b...,NaN,yes,NaN,"[-0.03081938996911049, -0.009769835509359837, ..."
4,39900,output.id: 089e9d67-46e7-4ac6-88cf-23cba2d7cfa...,Who should I contact for my car configuration ...,yes,NaN,"[-0.013012833893299103, 0.0025310523342341185,..."


In [ ]:
def insert_documents(df: pd.DataFrame):
    for index, row in df.iterrows():
        print(f"Inserting document with ID: {int(row['id'])}") # VH Previously: print(f"Inserting document with ID: {int(row['id']*10)}")
        data = {
            "id": int(row["id"]), # VH Previously: "id": int(row["id"]*10),
            "content": row["content"],
            "metadata": row.get("metadata", None),
            "embedding": row["embedding"]
        }
        response = supabase_client.table("retrieval_grammer").upsert(data).execute() # VH table name should correspond exactly with given name in supabase / changed insert to upsert



In [ ]:
insert_documents(df)

In [ ]:
def get_embedding(text):
    """Get embeddings for a text using OpenAI's embedding model"""
    embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY,model="text-embedding-3-large")  # Use the appropriate model
    response = embeddings.embed_query(text)  # Correctly call the method to generate embeddings
    embedding = response
    return embedding

In [ ]:
#Connect to Supabase
 ### Supabase Credentials
SUPABASE_URL = ""
SUPABASE_KEY = supabase_key


# Create Supabase client
supabase = create_client(SUPABASE_URL, SUPABASE_KEY)

In [ ]:
# -------------------------------
# 3. Define Retrieval Function for Multiple Models with Re-Ranking
# -------------------------------

def retrieve(query, model_name, k=10, re_rank=False):
    """
    Retrieve top-k documents for a query using Supabase vector search with optional LLM re-ranking.

    Parameters:
      query (str): The search query.
      model_name (str): One of 'gpt-4o-mini', 'claude', 'mistral', etc.
      k (int): Number of top documents to retrieve.
      re_rank (bool): Whether to re-rank the documents using the LLM.

    Returns:
      List[dict]: A list of dictionaries with document 'id', 'rank', and 'content'.
    """

    # Initialize LLM
    model_name = model_name.lower()
    if model_name == "gpt":
        llm = ChatOpenAI(model_name="gpt-4o-2024-11-20", openai_api_key=OPENAI_API_KEY)
    elif model_name == "claude":
        llm = ChatAnthropic(model="claude-3-7-sonnet-latest", anthropic_api_key=CLAUDE_API_KEY)
    elif model_name == "mistral":
        llm = ChatMistralAI(model="mistral-large-latest", mistral_api_key=MISTRAL_API_KEY)
    elif model_name == "cohere":
        llm = ChatCohere(model="command-a-03-2025", cohere_api_key=COHERE_API_KEY)
    elif model_name == "deepseek":
        import os
        os.environ["DEEPSEEK_API_KEY"] = DEEPSEEK_API_KEY
        llm = ChatDeepSeek(model="deepseek-v3-chat")
    else:
        raise ValueError(f"Unsupported model: {model_name}")

    try:
        # Step 1: Get embedding
        query_embedding = get_embedding(query)

        # Step 2: Retrieve documents from Supabase via RPC
        response = supabase.rpc(
            'match_documents_grammer_no_filter',
            {
                'query_embedding': query_embedding,
                'match_count': k
            }
        ).execute()

        if not response.data or len(response.data) == 0:
            print("No relevant documents found.")
            return []

        # Step 3: Build document list
        docs = [
            type("Doc", (object,), {
                "id": doc.get("id"),
                "page_content": doc.get("content", ""),
                "metadata": doc.get("metadata", {})
            })()
            for doc in response.data
        ]

        actual_k = min(k, len(docs))

        # Step 4: Optional re-ranking
        if re_rank and actual_k > 1:
            try:
                documents_text = "\n\n".join([
                    f"Document {i+1} (ID: {docs[i].id}):\n{docs[i].page_content}"
                    for i in range(actual_k)
                ])

                prompt = f"""
                Query: {query}

                You will be given {actual_k} documents retrieved via semantic search.
                Your task is to re-rank these documents in order of their relevance to the query.
                Please return EXACTLY {actual_k} document numbers in order, from MOST to LEAST relevant,
                separated by commas (e.g., "3,1,2").

                Documents:
                {documents_text}
                """

                messages = [
                    SystemMessage(content="You are a helpful assistant skilled at ranking document relevance."),
                    HumanMessage(content=prompt)
                ]

                llm_response = llm.invoke(messages)
                ranking_text = llm_response.content.strip()
                ranking_order = [int(num.strip()) - 1 for num in re.findall(r'\d+', ranking_text)]

                if len(ranking_order) != actual_k or sorted(ranking_order) != list(range(actual_k)):
                    print(f"Invalid ranking received: {ranking_text}. Using default order.")
                    ranking_order = list(range(actual_k))

                docs = [docs[i] for i in ranking_order]

            except Exception as e:
                print(f"Re-ranking failed: {e}. Using initial ranking.")

        # Step 5: Return formatted result
        results = [
            {
                "id": doc.id,
                "rank": idx + 1,
                "content": doc.page_content
            }
            for idx, doc in enumerate(docs)
        ]
        return results

    except Exception as e:
        print(f"Error retrieving documents: {e}")
        return []

In [ ]:
# VH For display in next cell
from IPython.display import display, HTML

In [ ]:
# Example usage (uncomment to test):
results_example = retrieve("What are the cafeteria plan benefits?", "gpt", k=2, re_rank=True)
# print(results_example)
display(HTML(f"<pre style='white-space:pre-wrap;'>{results_example}</pre>")) # VH

<ipython-input-7-ce6593dd82d4>:22: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4o-2024-11-20", openai_api_key=OPENAI_API_KEY)


In [ ]:
# -------------------------------
# 4. Prepare 50 Test Queries          # VH it's 39 now
# -------------------------------
test_queries = [
    "How can I access my Officient Calendar?", # 3986
    "What are the cafeteria plan benefits?", # 3989
    "What about my car configuration offer?", # 3990
    "What about Connecting Expertise?", # 3995
    "What's the BeCentral address?", # 3996
    "What about a Microsoft 365 license?", # 3997
    "What about Google Calendar", # 3998
    "What are compensatory rest days?", # 4000
    "How do I access the shared library?", # 4001
    "What is the login for StaffIT?", # 4003
    "How can I export contacts from Odoo?", # 4005
    "How can I export leads from Odoo?", # 4006
    "What is the structure for OneDrive?", # 4007
    "Who is responsible in case of a traffic fine?", # 4008
    "What about mailing lists?", # 4011
    "What about a green card?", # 4014
    "What about the company credit card?", # 4016
    "How to create a teams meeting from Google Agenda?", # 4017
    "What about Supplementary Family Allowances?", # 4018
    "On what days does the company post on LinkedIn?", # 4019
    "What activities are included in the DTeam Spirit Challenge?", # 4020
    "What are the limits for the mobility budget?", # 4021
    "What about Nexxtmove?", # 4024
    "How to use Odoo for CRM?", # 4025
    "What about Officient employee self-service?", # 4026
    "What about the Onboarding To Do List?", # 4027
    "What about birth leave?", # 4028
    "What about dtsc.odoo.com?", # 4030
    "What about ProUnity?", # 4031
    "What about a hiring bonus?", # 4032
    "What about Powerdale?", # 4034
    "What about Single Permits?", # 4035
    "What about the BNP application?", # 4037
    "Who are our suppliers?", # 4041
    "What is TED?", # 4042
    "How to activate Music Streaming?", # 4043
    "What is Scrum for?", # 4046
    "How to add a Shared Mailbox?", # 4047
    "What about BNP Paribas warrants?" # 4048
]

In [ ]:
# Ensure exactly 50 queries.      # VH it's 39 now
if len(test_queries) < 39:
    test_queries = test_queries * (39 // len(test_queries) + 1)
test_queries = test_queries[:39]
print("Prepared {} test queries.".format(len(test_queries)))

Prepared 39 test queries.


In [ ]:
test_queries

In [ ]:
# -------------------------------
# 5. Retrieve Documents for Each Query Across All Models
# -------------------------------
# Initialize a list to collect data

models = ["gpt-4o-mini",  "claude", "mistral", "cohere", "deepseek"]
# retrieval_results structure: { model_name: { query: [list of document results] } }
retrieval_results = {model: {} for model in models}

for model in models:
    for query in test_queries:
        # Set re_rank=True if you wish to re-rank documents using the LLM.
        retrieval_results[model][query] = retrieve(query, model, k=2, re_rank=True, search_method="keyword")
print("Retrieval complete for all models and queries.")

In [ ]:

# Initialize a list to collect data
data = []

# Iterate over each model and its corresponding queries
for model, queries in retrieval_results.items():
    for query, documents in queries.items():
        for doc in documents:
            data.append((model, query, doc['rank'], doc['id']))


# Create a DataFrame from the collected data
df = pd.DataFrame(data, columns=['Model', 'Query', 'Rank', 'Document ID'])


In [ ]:
df

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Read the ORIGINAL Document for getting the categories
df_original = pd.read_csv('/content/drive/My Drive/Internship DTSC/Dataset Grammar Bias 0404.csv')
df_original['id'] = df_original.id

In [ ]:
df_original.head()

,id,content,test_prompt,spelling_mistakes,comment
0,39860,output.id: 368d3765-a38d-4f63-bffe-0f59d86be5d...,How can I synchronize my Officient Calendar wi...,no,NaN
1,39861,output.id: 368d3765-a38d-4f63-bffe-0f59d86be5d...,NaN,yes,NaN
2,39890,output.id: e849b293-ce88-42bb-8a3c-6d7d88e459b...,What are the cafeteria plan benefits?,no,NaN
3,39891,output.id: e849b293-ce88-42bb-8a3c-6d7d88e459b...,NaN,yes,NaN
4,39900,output.id: 089e9d67-46e7-4ac6-88cf-23cba2d7cfa...,Who should I contact for my car configuration ...,yes,NaN


In [ ]:
df_category = pd.merge(df, df_original, right_on='id',left_on='Document ID')

In [ ]:

# Set up the plot grid: 1x2 for ranks 1 and 2
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
#fig.suptitle("Grammatical Error Category Distribution by Model for Each Rank", fontsize=16)

# Loop through Rank 1 to 2


subset = df[df["Rank"] == 1]
sns.countplot(data=subset, x="spelling_mistakes", hue="Model", order=["yes", "no"], ax=ax, palette=model_palette)
ax.set_title(f"Rank {rank}")
ax.set_xlabel("Error Category")
ax.set_ylabel("Count")
ax.legend(title="Model")

# Improve layout
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()
fig.savefig("retrieval_grammer_distribution.png", dpi=300, bbox_inches='tight')

In [ ]:
#CHI SQUARE TEST
# Define expected category by rank
expected_mapping = {
    1: "no",
    2: "yes",
}

# Add a column to check if each row matches the expected category for its rank
df["is_correct"] = df.apply(lambda row: row["spelling_mistakes"] == expected_mapping.get(row["Rank"]), axis=1)

# Create a contingency table: Model vs Correctness
contingency = pd.crosstab(df["Model"], df["is_correct"])

# Print contingency table
print("Contingency Table (Correct vs Incorrect by Model):")
print(contingency)

# Perform chi-square test
chi2, p, dof, expected = chi2_contingency(contingency)

# Print results
print("\nChi-Square Test Results:")
print(f"Chi2 Statistic: {chi2}")
print(f"Degrees of Freedom: {dof}")
print(f"P-value: {p}")

# Interpretation
alpha = 0.05
if p < alpha:
    print("Result: Statistically significant difference in correctness between models.")
else:
    print("Result: No statistically significant difference in correctness between models.")

In [ ]:

# Expected mapping of rank to date_category
expected_mapping = {1: "no", 2: "yes"}
category_to_rank = {"no": 1, "yes": 2}

# Add evaluation columns
df["is_correct"] = df.apply(lambda row: row["spelling_mistakes"] == expected_mapping.get(row["Rank"]), axis=1)
df["expected_rank"] = df["spelling_mistakes"].map(category_to_rank)

# 1. Accuracy (%)
accuracy_by_model = df.groupby("Model")["is_correct"].mean() * 100

# 2. Mean Absolute Error
mae_by_model = df.groupby("Model").apply(lambda x: mean_absolute_error(x["Rank"], x["expected_rank"]))

# 3. Cohen's Kappa
kappa_by_model = df.groupby("Model").apply(lambda x: cohen_kappa_score(x["Rank"], x["expected_rank"]))

# 4. Chi-Square Test of Correctness Distribution
contingency = pd.crosstab(df["Model"], df["is_correct"])
chi2, p_value, dof, expected = chi2_contingency(contingency)

# 5. Compile summary table
summary_table = pd.DataFrame({
    "Accuracy (%)": accuracy_by_model.round(2),
    "Mean Absolute Error": mae_by_model.round(2),
    "Cohen's Kappa": kappa_by_model.round(2)
})

# Display results
print("\n--- Model Comparison Table ---")
print(summary_table)

print("\n--- Chi-Square Test of Correctness Distribution ---")
print("Contingency Table:")
print(contingency)
print(f"Chi2 Statistic: {chi2:.4f}")
print(f"Degrees of Freedom: {dof}")
print(f"P-Value: {p_value:.4f}")

# Interpretation
alpha = 0.05
if p_value < alpha:
    print("→ Statistically significant difference in correctness between models.")
else:
    print("→ No statistically significant difference between models.")

In [ ]:
 #--- Initialize confusion metrics storage ---
confusion_metrics = []

# Loop through each model and compute a confusion matrix
for model in df["Model"].unique():
    model_df = df[df["Model"] == model]

    # Compute confusion matrix for the current model
    conf_matrix = confusion_matrix(model_df["Rank"], model_df["expected_rank"])

    # Extract True Positives, False Positives, False Negatives, True Negatives for each class
    tp = conf_matrix.diagonal()  # True Positives (diagonal elements)
    fp = conf_matrix.sum(axis=0) - tp  # False Positives (sum of columns - diagonal)
    fn = conf_matrix.sum(axis=1) - tp  # False Negatives (sum of rows - diagonal)
    tn = conf_matrix.sum() - (tp + fp + fn)  # True Negatives (sum of all - TP, FP, FN)

    # Calculate Precision, Recall, Specificity for each class
    precision = tp / (tp + fp)  # Precision = TP / (TP + FP)
    recall = tp / (tp + fn)  # Recall (Sensitivity) = TP / (TP + FN)
    specificity = tn / (tn + fp)  # Specificity = TN / (TN + FP)
    f1_score = 2 * (precision * recall) / (precision + recall)  # F1 Score

    # Append the metrics to the confusion_metrics list
    for i, rank in enumerate(expected_mapping.values()):
        confusion_metrics.append({
            "Model": model,
            "Rank": rank,
            "True Positives": tp[i],
            "False Positives": fp[i],
            "False Negatives": fn[i],
            "True Negatives": tn[i],
            "Precision": precision[i],
            "Recall (Sensitivity)": recall[i],
            "Specificity": specificity[i],
            "F1 Score": f1_score[i]
        })

# Convert the confusion metrics into a DataFrame for a nice tabular presentation
confusion_metrics_df = pd.DataFrame(confusion_metrics)

# Display the confusion matrix metrics table
print("\n--- Confusion Matrix Metrics by Model ---")
print(confusion_metrics_df)